In [1]:
import h5py
import numpy
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

C:\Users\Rakshith\Miniconda3\envs\machinelearning\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# check for GPU 
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17722081550549599760
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6707519816
locality {
  bus_id: 1
}
incarnation: 15295732122441999182
physical_device_desc: "device: 0, name: GeForce GTX 1070, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [3]:
# load ascii text and covert to lowercase
filename = "dataset/lovelyrics.txt"
raw_text = open(filename,encoding='utf-8').read()
raw_text = raw_text.lower()
raw_text

"the sun goes down\nthe stars come out\nand all that counts\nis here and now\nmy universe will never be the same\ni'm glad you came\nyou cast a spell on me, spell on me\nyou hit me like the sky fell on me, fell on me\nand i decided you look well on me, well on me\nso let's go somewhere no one else can see, you and me\nturn the lights out now\nnow i'll take you by the hand\nhand you another drink\ndrink it if you can\ncan you spend a little time,\ntime is slipping away,\naway from us so stay,\nstay with me i can make,\nmake you glad you came\nthe sun goes down\nsometimes i'm in disbelief i didn't know\nsomehow i need you to go\ndon't stay\nforget our memories\nforget our possibilities\nwhat you were changing me into\n(just give me myself back and)\ndon't stay\nforget our memories\nforget our possibilities\ntake all your faithlessness with you\njust give me myself back and\ndon't stay\nsometimes i feel like i trusted you too well\nsometimes i just feel like screaming at myself\nsometimes

In [4]:
# get all the chars in the sequence of text

chars = sorted(list(set(raw_text)))

In [5]:
# convert to dicts 

char_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_char = {i:ch for i,ch in enumerate(chars)}

In [6]:
# setup the variables used later 

n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters: {} ".format(n_chars))
print ("Total Vocab: {}".format(n_vocab))

Total Characters: 7437 
Total Vocab: 33


In [7]:
# prepare the dataset of input to output pairs encoded as integers

seq_length = 100
dataX = []
dataY = []

for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])

n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  7337


In [8]:
dataY

[16,
 13,
 1,
 26,
 9,
 21,
 13,
 0,
 17,
 2,
 21,
 1,
 15,
 20,
 9,
 12,
 1,
 32,
 23,
 28,
 1,
 11,
 9,
 21,
 13,
 0,
 32,
 23,
 28,
 1,
 11,
 9,
 26,
 27,
 1,
 9,
 1,
 26,
 24,
 13,
 20,
 20,
 1,
 23,
 22,
 1,
 21,
 13,
 5,
 1,
 26,
 24,
 13,
 20,
 20,
 1,
 23,
 22,
 1,
 21,
 13,
 0,
 32,
 23,
 28,
 1,
 16,
 17,
 27,
 1,
 21,
 13,
 1,
 20,
 17,
 19,
 13,
 1,
 27,
 16,
 13,
 1,
 26,
 19,
 32,
 1,
 14,
 13,
 20,
 20,
 1,
 23,
 22,
 1,
 21,
 13,
 5,
 1,
 14,
 13,
 20,
 20,
 1,
 23,
 22,
 1,
 21,
 13,
 0,
 9,
 22,
 12,
 1,
 17,
 1,
 12,
 13,
 11,
 17,
 12,
 13,
 12,
 1,
 32,
 23,
 28,
 1,
 20,
 23,
 23,
 19,
 1,
 30,
 13,
 20,
 20,
 1,
 23,
 22,
 1,
 21,
 13,
 5,
 1,
 30,
 13,
 20,
 20,
 1,
 23,
 22,
 1,
 21,
 13,
 0,
 26,
 23,
 1,
 20,
 13,
 27,
 2,
 26,
 1,
 15,
 23,
 1,
 26,
 23,
 21,
 13,
 30,
 16,
 13,
 25,
 13,
 1,
 22,
 23,
 1,
 23,
 22,
 13,
 1,
 13,
 20,
 26,
 13,
 1,
 11,
 9,
 22,
 1,
 26,
 13,
 13,
 5,
 1,
 32,
 23,
 28,
 1,
 9,
 22,
 12,
 1,
 21,
 13,
 0,
 27,
 28,
 25,
 22,

In [9]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [10]:
# define the LSTM model

model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [11]:
# define the checkpoint

filepath="love-songs-weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [12]:
# load the model from previous epoch - 1ooth epoch
# load the network weights
# filename = "eminem-weights-improvement-13-2.2727.hdf5"
# model.load_weights(filename)
# model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
import time 
# your code here

t1 = time.time()

with tf.device('/gpu:0'):
    model.fit(X, y, epochs=500, batch_size=128,callbacks=callbacks_list)

t2 = time.time()
print("Training took:")
t2-t1

Epoch 1/500
7337/7337 [==============================] - 34s - loss: 3.0802    
Epoch 2/500
7337/7337 [==============================] - 19s - loss: 3.0060    
Epoch 3/500
7337/7337 [==============================] - 13s - loss: 3.0047    
Epoch 4/500
7337/7337 [==============================] - 13s - loss: 2.9975    
Epoch 5/500
7337/7337 [==============================] - 13s - loss: 2.9844    
Epoch 6/500
7337/7337 [==============================] - 13s - loss: 2.9405    
Epoch 7/500
7337/7337 [==============================] - 13s - loss: 2.8622    
Epoch 8/500
7337/7337 [==============================] - 13s - loss: 2.7874    
Epoch 9/500
7337/7337 [==============================] - 13s - loss: 2.7228    
Epoch 10/500
7337/7337 [==============================] - 13s - loss: 2.6716    
Epoch 11/500
7337/7337 [==============================] - 13s - loss: 2.6217    
Epoch 12/500
7337/7337 [==============================] - 13s - loss: 2.5715    
Epoch 13/500
7337/7337 [=============

7337/7337 [==============================] - 13s - loss: 0.5453    
Epoch 33/500
7337/7337 [==============================] - 13s - loss: 0.4719    
Epoch 34/500
7337/7337 [==============================] - 13s - loss: 0.4343    
Epoch 35/500
7337/7337 [==============================] - 13s - loss: 0.3943    
Epoch 36/500
7337/7337 [==============================] - 13s - loss: 0.3567    
Epoch 37/500
7337/7337 [==============================] - 13s - loss: 0.3135    
Epoch 38/500
7337/7337 [==============================] - 13s - loss: 0.2903    
Epoch 39/500
7337/7337 [==============================] - 13s - loss: 0.2537    
Epoch 40/500
7337/7337 [==============================] - 13s - loss: 0.2299    
Epoch 41/500
7337/7337 [==============================] - 14s - loss: 0.2119    
Epoch 42/500
3456/7337 [=============>................] - ETA: 7s - loss: 0.1834

In [36]:
import sys

# load text from the model

# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Love song Lyrics Starting now:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

# generate characters
for i in range(3000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index) 
    pattern = pattern[1:len(pattern)]
print ("\n Hope it dropped cool words ;) ")

Rap Lyric Starting now:
" ting my sales on billboards yeah i shine like gold
y'all clusters i'm taking over the universe y'all "
 got them fake converses
it's so worthless for y'all cause to verse jordeezy and youngdeezy
cause they both go hard as shit jordeezy been dead and gone
youngdeezy taken over his soul yeah i'm an asshole but girls love it
im blasting shit
bitches on my johnson no magic shit
niggas talking crazy about my city i ain't having it
talking about new york fell off, ain't that some shit
if i ever see us falling off i bet im catching it
paper on your head through a phone call im faxing shit
oh and i get a quarter back and i sack a bitch
adding bitches subtracting bitches that mathamatic shitt
run up on your block and u git shocked when that static hit
im acting up, u r l smack it up
now u mad as fuck. you niggas softer than saggy nuts
im stacking up this dough hotter than dragon butt
my flow catching up you ugly couldn't bag a slut
i was never used to
being what i used